## Trabalho Análise de Modelos Preditivos com Sklearn

Aluno: Leonardo Henrique de Figueiredo


Modelo de Classificacao - Regressao logistica

Dataset: Venda de bicicletas na Europa. https://www.kaggle.com/code/sadiqshah/bike-store-sales-in-europe/data

In [1]:
#importacao de dependencias 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve, classification_report,\
                            accuracy_score, confusion_matrix, auc

In [3]:
# descompactacao de arquivo 
import zipfile

with zipfile.ZipFile('/home/sklearn/Trabalho/Sales.csv.zip', 'r') as zipref:
    zipref.extractall('/home/sklearn/Trabalho/Sales_bike')

In [3]:
# leitura do dataset --- alterado nome e caminho para sua avaliacao 
#sales = pd.read_csv("/home/sklearn/Trabalho/Sales_bike/Sales.csv")
sales = pd.read_csv("/home/Trabalho/pasta_dataset2/dataset2.csv") 

In [82]:
sales.head(10)

,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418
5,2016-05-15,15,May,2016,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,5,45,120,297,225,522
6,2014-05-22,22,May,2014,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,199,180,379
7,2016-05-22,22,May,2016,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,2,45,120,100,90,190
8,2014-02-22,22,February,2014,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,22,45,120,1096,990,2086
9,2016-02-22,22,February,2016,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,21,45,120,1046,945,1991


In [83]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113036 entries, 0 to 113035
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Date              113036 non-null  object
 1   Day               113036 non-null  int64 
 2   Month             113036 non-null  object
 3   Year              113036 non-null  int64 
 4   Customer_Age      113036 non-null  int64 
 5   Age_Group         113036 non-null  object
 6   Customer_Gender   113036 non-null  object
 7   Country           113036 non-null  object
 8   State             113036 non-null  object
 9   Product_Category  113036 non-null  object
 10  Sub_Category      113036 non-null  object
 11  Product           113036 non-null  object
 12  Order_Quantity    113036 non-null  int64 
 13  Unit_Cost         113036 non-null  int64 
 14  Unit_Price        113036 non-null  int64 
 15  Profit            113036 non-null  int64 
 16  Cost              113036 non-null  int

Agora vamos estimar uma regressão logística 



In [84]:
#por categoria, filtro somente Bike para o trabalho
sales.Product_Category.unique()

array(['Accessories', 'Clothing', 'Bikes'], dtype=object)

In [85]:
#definicao de coluna Product_Category como tipo category
sales["Product_Category"] = sales["Product_Category"].astype("category")

In [86]:
# removendo da coluna Product category as categorias Accessories e Clothing
sales["Product_Category"] = sales["Product_Category"].cat.remove_categories("Accessories")

In [87]:
sales["Product_Category"] = sales["Product_Category"].cat.remove_categories("Clothing")

In [88]:
#limpando os NA's
salesbike=sales.dropna(subset=['Product_Category'])

In [89]:
salesbike.head()

,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
49304,2013-01-28,28,January,2013,31,Young Adults (25-34),M,Australia,Queensland,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,1912,3400,944,1912,2856
49305,2015-01-28,28,January,2015,31,Young Adults (25-34),M,Australia,Queensland,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,1912,3400,944,1912,2856
49306,2013-07-22,22,July,2013,31,Young Adults (25-34),M,Australia,Queensland,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,1912,3400,944,1912,2856
49307,2015-07-22,22,July,2015,31,Young Adults (25-34),M,Australia,Queensland,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",2,1912,3400,1888,3824,5712
49308,2013-12-25,25,December,2013,31,Young Adults (25-34),M,Australia,Queensland,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,1912,3400,944,1912,2856


In [90]:
salesbike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25982 entries, 49304 to 112071
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Date              25982 non-null  object  
 1   Day               25982 non-null  int64   
 2   Month             25982 non-null  object  
 3   Year              25982 non-null  int64   
 4   Customer_Age      25982 non-null  int64   
 5   Age_Group         25982 non-null  object  
 6   Customer_Gender   25982 non-null  object  
 7   Country           25982 non-null  object  
 8   State             25982 non-null  object  
 9   Product_Category  25982 non-null  category
 10  Sub_Category      25982 non-null  object  
 11  Product           25982 non-null  object  
 12  Order_Quantity    25982 non-null  int64   
 13  Unit_Cost         25982 non-null  int64   
 14  Unit_Price        25982 non-null  int64   
 15  Profit            25982 non-null  int64   
 16  Cost             

In [92]:
salesbike = salesbike.drop(['Day','State','Unit_Cost', 'Unit_Price'], axis=1)

In [94]:
#Irei utilizar somente a categoria Bike e suas subcategorias
salesbike.Sub_Category.unique()

array(['Mountain Bikes', 'Road Bikes', 'Touring Bikes'], dtype=object)

In [95]:
salesbike['Sub_Category'] = salesbike['Sub_Category'].astype('category') # transformando Sub_Category para categórico

salesbike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25982 entries, 49304 to 112071
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Date              25982 non-null  object  
 1   Month             25982 non-null  object  
 2   Year              25982 non-null  int64   
 3   Customer_Age      25982 non-null  int64   
 4   Age_Group         25982 non-null  object  
 5   Customer_Gender   25982 non-null  object  
 6   Country           25982 non-null  object  
 7   Product_Category  25982 non-null  category
 8   Sub_Category      25982 non-null  category
 9   Product           25982 non-null  object  
 10  Order_Quantity    25982 non-null  int64   
 11  Profit            25982 non-null  int64   
 12  Cost              25982 non-null  int64   
 13  Revenue           25982 non-null  int64   
dtypes: category(2), int64(6), object(6)
memory usage: 2.6+ MB


In [96]:
salesbike.Age_Group.unique()

array(['Young Adults (25-34)', 'Youth (<25)', 'Adults (35-64)',
       'Seniors (64+)'], dtype=object)

In [97]:
#Definindo grupos por idade
salesbike['Customer_Gender'] = salesbike['Customer_Gender'] .map({'M': 1, 'F': 0})
salesbike['Age_Group'] = salesbike['Age_Group'] .map({'Youth (<25)': 1, 'Young Adults (25-34)': 0, 'Adults (35-64)':0, 'Seniors (64+)':0})

In [112]:
salesbike.head()

,Date,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,Product_Category,Sub_Category,Product,Order_Quantity,Profit,Cost,Revenue
49304,2013-01-28,January,2013,31,0,1,Australia,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,944,1912,2856
49305,2015-01-28,January,2015,31,0,1,Australia,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,944,1912,2856
49306,2013-07-22,July,2013,31,0,1,Australia,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,944,1912,2856
49307,2015-07-22,July,2015,31,0,1,Australia,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",2,1888,3824,5712
49308,2013-12-25,December,2013,31,0,1,Australia,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,944,1912,2856


In [113]:
#Verificando os paises
salesbike.Country.unique()

array(['Australia', 'United States', 'Canada', 'France', 'United Kingdom',
       'Germany'], dtype=object)

In [114]:
model = LogisticRegression(penalty='none', solver='lbfgs') #newton-cg - método de resolução

In [147]:
salescnt = salesbike[['Country', 'Customer_Gender','Age_Group', 'Sub_Category','Profit']]


In [148]:
salescnt.head()

,Country,Customer_Gender,Age_Group,Sub_Category,Profit
49304,Australia,1,0,Mountain Bikes,944
49305,Australia,1,0,Mountain Bikes,944
49306,Australia,1,0,Mountain Bikes,944
49307,Australia,1,0,Mountain Bikes,1888
49308,Australia,1,0,Mountain Bikes,944


In [149]:
X = pd.get_dummies(salescnt[['Country', 'Age_Group', 'Sub_Category','Profit']]) # transformação em variável dummy

In [150]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25982 entries, 49304 to 112071
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age_Group                    25982 non-null  int64
 1   Profit                       25982 non-null  int64
 2   Country_Australia            25982 non-null  uint8
 3   Country_Canada               25982 non-null  uint8
 4   Country_France               25982 non-null  uint8
 5   Country_Germany              25982 non-null  uint8
 6   Country_United Kingdom       25982 non-null  uint8
 7   Country_United States        25982 non-null  uint8
 8   Sub_Category_Mountain Bikes  25982 non-null  uint8
 9   Sub_Category_Road Bikes      25982 non-null  uint8
 10  Sub_Category_Touring Bikes   25982 non-null  uint8
dtypes: int64(2), uint8(9)
memory usage: 837.3 KB


In [151]:
X

,Age_Group,Profit,Country_Australia,Country_Canada,Country_France,Country_Germany,Country_United Kingdom,Country_United States,Sub_Category_Mountain Bikes,Sub_Category_Road Bikes,Sub_Category_Touring Bikes
49304,0,944,1,0,0,0,0,0,1,0,0
49305,0,944,1,0,0,0,0,0,1,0,0
49306,0,944,1,0,0,0,0,0,1,0,0
49307,0,1888,1,0,0,0,0,0,1,0,0
49308,0,944,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
112067,0,2420,0,0,0,0,1,0,0,0,1
112068,0,807,0,0,0,0,1,0,0,0,1
112069,0,1613,0,0,0,0,1,0,0,0,1
112070,0,807,0,0,0,0,1,0,0,0,1


In [152]:
X.columns = X.columns.str.replace(" ", "_")

In [153]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25982 entries, 49304 to 112071
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age_Group                    25982 non-null  int64
 1   Profit                       25982 non-null  int64
 2   Country_Australia            25982 non-null  uint8
 3   Country_Canada               25982 non-null  uint8
 4   Country_France               25982 non-null  uint8
 5   Country_Germany              25982 non-null  uint8
 6   Country_United_Kingdom       25982 non-null  uint8
 7   Country_United_States        25982 non-null  uint8
 8   Sub_Category_Mountain_Bikes  25982 non-null  uint8
 9   Sub_Category_Road_Bikes      25982 non-null  uint8
 10  Sub_Category_Touring_Bikes   25982 non-null  uint8
dtypes: int64(2), uint8(9)
memory usage: 837.3 KB


In [154]:
#Contagem de Age_Group - por idade
y = salescnt.Customer_Gender
y

49304     1
49305     1
49306     1
49307     1
49308     1
         ..
112067    0
112068    0
112069    0
112070    0
112071    0
Name: Customer_Gender, Length: 25982, dtype: int64

In [155]:
model.fit(X,y)

LogisticRegression(penalty='none')

In [156]:
print(model.coef_)

[[ 2.44098635e-01 -2.48473101e-05  2.60647140e-02  1.21993129e-02
   5.87356246e-02 -1.26630745e-01  4.66220468e-02 -2.02726981e-02
  -4.70306970e-02  4.82812272e-02 -4.53227547e-03]]


-1,73 (sex - b1)     -0,069(age - b2) + 0,33 (retecg - b3) -0,70 (cp_0 - b4) + 1,77 (cp_1 - b5) + 1,64 (cp_2 - b6) + 1,60(cp_3 - b7)   

y(estimativa) = B0 -1,73*sex -0,069*age +0,33*retecg -0,70*cp_0 +1,77*cp_1 +1,64*cp_2 +1,60*cp_3                   #Sinal menos ---- diminui    

Rodar modelo de regressão linear generalizada

In [157]:
modelo = smf.glm(formula='Age_Group ~ Profit + Country +  Sub_Category', data=salescnt, 
                 family = sm.families.Binomial()).fit()

In [158]:
modelo

In [159]:
print(modelo.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Age_Group   No. Observations:                25982
Model:                            GLM   Df Residuals:                    25973
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -9818.9
Date:                Wed, 26 Oct 2022   Deviance:                       19638.
Time:                        20:08:18   Pearson chi2:                 2.59e+04
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01915
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [160]:
yestimativa=model.predict_proba(X)

In [161]:
yestimativa # calcula por linha a probabilidade 

array([[0.51192364, 0.48807636],
       [0.51192364, 0.48807636],
       [0.51192364, 0.48807636],
       ...,
       [0.50031767, 0.49968233],
       [0.49531108, 0.50468892],
       [0.50031767, 0.49968233]])

In [162]:
confusion_matrix(y, model.predict(X)) # matriz comparação entre y (real) e model.predict(X)-predição do algoritmo ----é uma contagem de positivos e negativos

array([[6051, 6801],
       [5969, 7161]])

In [163]:
pd.crosstab(y, model.predict(X)) 


col_0,0,1
Customer_Gender,,
0,6051,6801
1,5969,7161


true positive   false positive

False negative  True negative 

In [164]:
kpis = (classification_report(y, model.predict(X))) 
print(kpis)

              precision    recall  f1-score   support

           0       0.50      0.47      0.49     12852
           1       0.51      0.55      0.53     13130

    accuracy                           0.51     25982
   macro avg       0.51      0.51      0.51     25982
weighted avg       0.51      0.51      0.51     25982



50%% de precisao

Até 0,50 - Ruim     entre 0,51 e 0,80 – Bom      acima de 0,81 - Confiável 
- ----- > Ruim!


Recall tem que ser alto.

Valores muito proximos de recall e precisao. Nao conseguimos reduzir o numero de falsos positivos 